In [ ]:
data={}
import csv
with open('fullcounty1.csv', 'r') as f:
    reader = csv.DictReader(f);
    
    for row in reader:
        if(row['population'] == '#N/A'):
            break
        population = int(row['population'])
        date = str(row['ï»¿date'])
        statefip = int(row['state_fips'])
        countyfip = int(row['county_fips'])
        cases = int(row['cases'])
        
        if statefip not in data:
            data[statefip] = {}
            print("create "+str(statefip))
        if countyfip not in data[statefip]:
            data[statefip][countyfip] = []
            
        newdict = {
            'date' : date,
            'cases': cases,
            'population':population,
        }
        
        (data[statefip])[countyfip].append(newdict)
            
    


In [ ]:
import json
with open('dataz.json', 'w') as f:
    json.dump(data, f)

In [ ]:
## Step 2: Load the shapes for the counties (features), lookup the labor force
## values based on the state/county codes, and set it as part of the feature
## properties
import pandas as pd
geojson1={"type":"FeatureCollection", "features": []}
df2 = pd.read_csv('bordercounty.csv', dtype=str)

with open('counties2.geojson', 'r') as f:
    geojson = json.load(f) 
    
    for feature in geojson['features']:
        featureProperties = feature['properties']
        statefp = int(featureProperties['STATEFP'])
        countyfp = int(featureProperties['COUNTYFP'])
        
        for indexborder, row in df2.iterrows():
            if (statefp == int(row['state_fip'])):
                
                if (countyfp == int(row['county_fip'])):
                    
                    if (data.get(statefp) is not None):
                    
                        temp = 0
                        previous = {
                            'cases':0,
                            'risk':0,
                        }
                        total = 0
                        indx = 0 # 2nd loop starter - 7d rolling
                        i=0 # first loop starter - 7d rolling
                        mylist=[]
                        go = 0 # 2nd if
                        for featureData in data.get(statefp).get(countyfp, {}):
                        
                            featurecopy = json.loads(json.dumps(feature))

                            geojson1['features'].append(featurecopy)


                            (((geojson1['features'])[-1])['properties'])['date'] = featureData['date']

                            if(i==7 and indx==7):
                                go = 1
                            if(i==6 and indx==6):
                                mylist.append(((featureData['cases']-previous['cases'])/featureData['population'])*100000)
                                moving = total/7
                                if(moving < 0):
                                    moving = 0
                                if(moving < 1):
                                    risk = 1
                                elif (moving < 10):
                                    risk = 2
                                elif (moving < 25):
                                    risk = 3
                                else:
                                    risk = 4
                                (((geojson1['features'])[-1])['properties'])['risk'] = risk  
                                (((geojson1['features'])[-1])['properties'])['rolling']=moving  
                                i += 1
                                indx += 1
                            if(i<6):
                                mylist.append(((featureData['cases']-previous['cases'])/featureData['population'])*100000) #(data[i+1] - data[i])
                                total+= mylist[indx]
                                i += 1
                                indx += 1  
                                (((geojson1['features'])[-1])['properties'])['rolling'] = 0 # less than 7 days
                                (((geojson1['features'])[-1])['properties'])['risk'] = 1 # green - on track to contain             
                            if(go==1):
                                mylist.append(((featureData['cases']-previous['cases'])/featureData['population'])*100000)
                                total = total - mylist[indx-7] +  mylist[-1] # total - (data[indx+1] - data[indx]) + (data[i+1] - data[i])
                                moving = total/7
                                if (moving < 0):
                                    moving = 0
                                if(moving < 1):
                                    risk = 1
                                elif (moving < 10):
                                    risk = 2
                                elif (moving < 25):
                                    risk = 3
                                else:
                                    risk = 4
                                (((geojson1['features'])[-1])['properties'])['risk'] = risk  
                                (((geojson1['features'])[-1])['properties'])['rolling']=moving
                                indx += 1
                             

                            previous = featureData
                            previous['risk'] = temp




In [ ]:

with open('us_rolling_avg.geojson', 'w') as f:
    json.dump(geojson1, f)

In [ ]:
import csv
import pandas as pd

df3 = pd.read_csv('fullcounty1.csv')
df3

In [ ]:
df2 = pd.read_csv('bordercounty.csv', dtype=str)
# total new cases in border counties everyday

df2

In [ ]:
fip=[]
for indx, row in df2.iterrows():
    fip.append(row['state_fip']+row['county_fip'])


In [ ]:
fip

In [ ]:
date=[]
statefip=[]
countyfip=[]
newcases=[]
state=[]
previousfp='1001'
pop=[]
for indx, row in df3.iterrows():
    if(len(str(row['county_fips']))==1):
        myfip=str(row['state_fips'])+'00'+str(row['county_fips'])
    elif(len(str(row['county_fips']))==2):
        myfip=str(row['state_fips'])+'0'+str(row['county_fips'])
    else:
        myfip=str(row['state_fips'])+str(row['county_fips'])
    if(myfip in fip):
        print(myfip)
        if(previousfp!=myfip):
            newcases.append(int(row['cases']))
        else:
            new=int(row['cases'])-previouscase
            newcases.append(new)
        date.append(row['date'])
        statefip.append(row['state_fips'])
        countyfip.append(row['county_fips'])
        state.append(row['state'])
        pop.append(int(row['population']))
        previouscase=int(row['cases'])
        previousfp=myfip
       
       

In [ ]:
col=['date','state','statefip','countyfip','newcases', 'population']
mydf = pd.DataFrame(columns = col)
mydf

In [ ]:
mydf['date']=date
mydf['state']=state
mydf['statefip']=statefip
mydf['countyfip']=countyfip
mydf['newcases']=newcases
mydf['population']=pop
mydf

In [ ]:
mydf2=mydf.sort_values(by=['date'])
mydf2

In [ ]:
mydf2.head(50)      

In [ ]:
mydf2.tail(50)

In [ ]:
previousdate='2/10/20 0:00'
sumz=0
mydate=[]
mysum=[]

for indx, row in mydf2.iterrows():
    if(row['date']!=previousdate):
        mydate.append(previousdate)
        mysum.append(sumz)
        sumz=row['newcases']
        previousdate=row['date']
    else:
        sumz += row['newcases']
mydate.append(previousdate)
mysum.append(sumz)

In [ ]:
cols=['date','total_new_cases']
mydf3 = pd.DataFrame(columns = cols)
mydf3

In [ ]:
mydf3['date']=mydate
mydf3['total_new_cases']=mysum
mydf3

In [ ]:
mydf3.tail(7)

In [ ]:
mydf3.to_csv('USborder_total.csv')

In [ ]:
mydf4=mydf.sort_values(by=['population'])
mydf4

In [ ]:
cal=0
for x in mydf4.population.unique():
    cal += x
print(cal) # total pop in US border counties 

In [ ]:
mydf4.population.unique()